In [4]:
import urllib.request
import gensim
from gensim.models import word2vec

import nltk.data
import re
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer 

import pandas as pd
from tqdm import tqdm 

import pymorphy2

tqdm.pandas()

In [5]:
df = pd.read_csv('data/hhparser_vacancy.csv')

In [6]:
def remove_tags(text: str) -> str:
    return re.sub('<[^>]*>', ' ', text)

In [7]:
df.description.fillna('none', inplace=True)

In [8]:
df.description = df.description.str.replace('ё', 'е')

In [10]:
descriptions = df.description.progress_apply(remove_tags)

100%|██████████████████████████████████| 96225/96225 [00:02<00:00, 38331.95it/s]


In [11]:
import importlib
import normalizer
normalizer = importlib.reload(normalizer)
from normalizer import normalize_sentences

In [12]:
normal_sents = normalize_sentences(descriptions)

100%|██████████████████████████████████| 96225/96225 [00:05<00:00, 18699.78it/s]


In [15]:
import joblib
import multiprocessing

In [18]:
cores = multiprocessing.cpu_count()

model = word2vec.Word2Vec(min_count=20,
                         window=3,
                         vector_size=300,
                         sample=6e-5, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         negative=20,
                         workers=cores-1)

In [26]:
model.build_vocab(normal_sents, progress_per=1000)

In [28]:
%%time
model.train(normal_sents, total_examples=model.corpus_count, epochs=30, report_delay=1)

(265598100, 557322810)

In [32]:
model.wv.most_similar('зарплата')

[('зп', 0.892952024936676),
 ('заработный', 0.8042138814926147),
 ('плата', 0.7912531495094299),
 ('доход', 0.7435174584388733),
 ('оклад', 0.7322441935539246),
 ('з', 0.6915128231048584),
 ('премия', 0.6601575613021851),
 ('бонус', 0.6316412687301636),
 ('оплата', 0.6313985586166382),
 ('вознаграждение', 0.6246646642684937)]

In [35]:
model.save('hh_word2vec.pickle')

In [99]:
def text2vec(text):
    vector = np.array([.0 for _ in range(model.vector_size)])
    count = 0

    for word in text.split():
        if word in model.wv:
            vector += model.wv[word]
            count += 1

    if count != 0:
        vector /= count

    return vector


In [103]:
vectors = np.zeros((df.shape[0], model.vector_size))

for i, name in enumerate(df.name):
    vectors[i, :] = text2vec(name)

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
def most_similar_vacancee(name, top_n=5):
    vector = text2vec(name)
    similarities = cosine_similarity([vector], vectors)
    similarities = [(sim, df.name.iloc[i]) for i, sim in enumerate(similarities[0])]
    similarities.sort(reverse=True)
    
    return similarities[:top_n]

In [149]:
most_similar_vacancee('разработка сайта')

[(1.0000000000000002, 'Стажер Sberseasons Мобильная разработка Android'),
 (1.0000000000000002,
  'Системный аналитик/ Бизнес-аналитик разработка высоконагруженного ПО'),
 (1.0000000000000002,
  'Разработчик С++, кроссплатформенная разработка сетевых сервисов'),
 (1.0000000000000002, 'Unity разработчик/Gamedev, разработка игр'),
 (1.0000000000000002, 'Product manager мобильная разработка')]

In [132]:
model.wv.most_similar('оператор')

[('диспетчер', 0.49753934144973755),
 ('колл', 0.4464859068393707),
 ('мерчендайзер', 0.4109043478965759),
 ('продавец', 0.39603227376937866),
 ('комплектовщик', 0.3930642306804657),
 ('телемаркетолог', 0.39092662930488586),
 ('волоконный', 0.3791011571884155),
 ('авторизированный', 0.3667648136615753),
 ('регистратор', 0.3634858727455139),
 ('сортировочный', 0.35270535945892334)]

In [88]:
model.save('hh_word2vec.pickle')

In [47]:
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_model
model = load_facebook_model("cc.ru.300.bin")

In [ ]:
model.build_vocab(normal_sents, update=True)

In [53]:
model.train(normal_sents, total_examples=len(normal_sents), epochs=5)

(24840978, 93054275)

In [55]:
model.wv.most_similar('зарплата')

[('брутто-зарплата', 0.9950622916221619),
 ('зарплата.', 0.9942271709442139),
 ('зарплатами', 0.992497980594635),
 ('зарплатах', 0.9919713735580444),
 ('зарплатам', 0.9917556047439575),
 ('.Зарплата', 0.9915549755096436),
 ('Зарплата', 0.9915105700492859),
 ('Зарплата.ру', 0.9913815855979919),
 ('делЗарплата', 0.9909665584564209),
 ('1Зарплата', 0.9906765222549438)]

In [54]:
model.save('hh_fastext.300.pickle')